In [2]:
\l C:\q\columbiaHdb\
\c 25 200

cleanData:{[dt]
 tbl: select from bin10 where date = dt;
 tbl: delete spread from tbl;
 update reverse fills reverse date, 
 reverse fills reverse mid, 
 reverse fills reverse vol, 
 reverse fills reverse adv 
 by id from tbl}

tbl: cleanData 2019.01.03

# Exercise 1

1. Implement a TWAP scheduling function.
2. Implement a VWAP scheduling function.
3. Implement a PoV scheduling function with 10\% participation rate.
4. For a given stock, simulate buying \$1 million using each trading schedule. Compare the three schedules.
5. Repeat Question 4 with a buy order of 10\% adv for the same stock. Comment on PoV and VWAP schedules when one has a perfect volume prediction.

## Exercise 1.1

In [3]:
TWAPsimschedule: {[trade; adv; size]
    timeunits: count trade;
    mytrade: timeunits#(size % timeunits);
    mytrade}

## Exercise 1.2

In [4]:
VWAPsimschedule: {[trade; adv; size]
    mytrade: size * (abs(trade) % adv);
    mytrade}

## Exercise 1.3

In [5]:
PoVsimschedule:{[trade; adv; size]
    mytrade: 0.1 * abs(trade);
    totaltrades: sums(mytrade);
    totaltrades: totaltrades & size;
    mytrade: deltas(totaltrades);
    mytrade}

## Exercise 1.4

In [6]:
tbl2: select from tbl where id = 0
size: 1000000;
tbl2: update TWAPschedule: TWAPsimschedule[trade; adv; size],
 VWAPschedule: VWAPsimschedule[trade; adv; size],
 PoVschedule: PoVsimschedule[trade; adv; size] from tbl2

## Exercise 1.5

In [7]:
size: 1000000;
tbl2: update TWAPschedule: TWAPsimschedule[trade; adv; 0.1 * last adv],
 VWAPschedule: VWAPsimschedule[trade; adv; 0.1 * last adv],
 PoVschedule: PoVsimschedule[trade; adv; 0.1 * last adv] from tbl2

# Exercise 2

In this Exercise, we assume that we historically submitted a $10\%$ TWAP buy order on every stock in our universe. Furthermore, assume the price impact model
\begin{equation*}
dI_t = -\beta I_t dt + 0.8\cdot \frac{\sigma}{\text{adv}} dQ_t
\end{equation*}
with $\log(2)/\beta = 60$ minutes.

1. Compute impact-adjusted prices across the stock universe.
2. Simulate new prices assuming we instead submitted TWAP buy orders of $20\%$ adv.
3. Simulate new prices assuming we instead submitted VWAP buy orders of $10\%$ adv.
4. Compute the historical strategy's average impact costs. Repeat for the TWAP order from Question 2 and the VWAP order from Question 3.
5. Compute the historical strategy's P\&L. Repeat for the TWAP order from Question 2 and the VWAP order from Question 3.

In [8]:
computeImpact : {[trade; adv; vol]
    / exponential decay constant per 10 seconds,
    / i.e. \beta * \delta t in the formula above
    h: 60;
    beta: (log 2) % (6 * h);
    lambda: 0.8;
    dailyVol: vol * sqrt 6 * 60 * 6.5;
    / initialize impact to be zero
    / drop the first elemetn from ema in the output
    (neg count trade)#ema[beta; 0, lambda * dailyVol * trade % beta * adv]}

## Exercise 2.1

In [9]:
tbl: update mytrade: TWAPsimschedule[trade; adv; 0.1*adv] by id from tbl
tbl: update I: computeImpact[mytrade; adv; vol] by id from tbl
tbl: update adjusted_mid: mid - I from tbl

## Exercise 2.2

In [10]:
tbl: update mytrade2: TWAPsimschedule[trade; adv; 0.2*adv] by id from tbl
tbl: update I2: computeImpact[mytrade2; adv; vol] by id from tbl
tbl: update mid2: adjusted_mid + I2 from tbl

## Exercise 2.3

In [11]:
tbl: update mytrade3: VWAPsimschedule[trade; adv; 0.1*adv] by id from tbl
tbl: update I3: computeImpact[mytrade3; adv; vol] by id from tbl
tbl: update mid3: adjusted_mid + I3 from tbl

## Exercise 2.4

In [12]:
tbl: update instantDeltaI: (mytrade * 0.8 * (vol * sqrt 6 * 60 * 6.5)) % adv,
 instantDeltaI2: (mytrade2 * 0.8 * (vol * sqrt 6 * 60 * 6.5)) % adv,
 instantDeltaI3: (mytrade3 * 0.8 * (vol * sqrt 6 * 60 * 6.5)) % adv by id from tbl

In [13]:
tbl: update mytradecost: (I - 0.5 * instantDeltaI) * mytrade,
 mytrade2cost: (I2 - 0.5 * instantDeltaI2) * mytrade2,
 mytrade3cost: (I3 - 0.5 * instantDeltaI3) * mytrade3 by id from tbl

In [14]:
tbl3: select impactcost: sum mytradecost,
 impactcost2: sum mytrade2cost,
 impactcost3: sum mytrade3cost by id from tbl

In [15]:
tbl4: select avgimpactcost: avg impactcost,
 avgimpactcost2: avg impactcost2,
 avgimpactcost3: avg impactcost3 from tbl3

## Exercise 2.5

In [16]:
deltas0:{first[x]-':x}

In [17]:
tbl: update deltaPaL: ((sums mytrade) * (deltas0 adjusted_mid)) - mytradecost,
 deltaPaL2: ((sums mytrade2) * (deltas0 adjusted_mid)) -mytrade2cost,
 deltaPaL3: ((sums mytrade3) * (deltas0 adjusted_mid)) -mytrade3cost by id from tbl

In [18]:
tbl5: select PaL: sum deltaPaL,
 PaL2: sum deltaPaL2,
 PaL3: sum deltaPaL3 by id from tbl

In [19]:
tbl6: select PaL: avg PaL,
 PaL2: avg PaL2,
 PaL3: avg PaL3 from tbl5